# Unit 12 - Tales from the Crypto

---


## 1. Sentiment Analysis

Use the [newsapi](https://newsapi.org/) to pull the latest news articles for Bitcoin and Ethereum and create a DataFrame of sentiment scores for each coin.

Use descriptive statistics to answer the following questions:
1. Which coin had the highest mean positive score?
2. Which coin had the highest negative score?
3. Which coin had the highest positive score?

In [1]:
# Initial imports
import os
import pandas as pd
from dotenv import load_dotenv
import nltk as nltk
nltk.download('vader_lexicon')  
from nltk.sentiment.vader import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()
from newsapi.newsapi_client import NewsApiClient
from datetime import datetime, timedelta

%matplotlib inline

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\ironcommander\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [2]:
# Read your api key environment variable
load_dotenv()



True

In [3]:
# Create a newsapi client
newsapi = NewsApiClient(api_key=os.environ["NEWS_API_KEY"])


In [4]:
current_date = pd.Timestamp(datetime.now(), tz="America/New_York").isoformat()
past_date = pd.Timestamp(datetime.now()- timedelta(30), tz="America/New_York").isoformat()


In [16]:
# Use newsapi client to get most relevant 20 headlines per day in the past month
# and then create a headline list and append the recent headlines
# and then also create a sentiments dataframe and append to it
def get_headlines(keyword):
    all_headlines = []
    all_dates = []   
    all_count = [] 
    date = datetime.strptime(current_date[:10], "%Y-%m-%d")
    end_date = datetime.strptime(past_date[:10], "%Y-%m-%d")
    print(f"Fetching news about '{keyword}'")
    print("*" * 30)
    while date > end_date:
        print(f"retrieving news from: {date}")
        articles = newsapi.get_everything(
            q=keyword,
            from_param=str(date)[:10],
            to=str(date)[:10],
            language="en",
            sort_by="relevancy",
            page=1,
        )
        headlines = []
        for i in range(0, len(articles["articles"])):
            headlines.append(articles["articles"][i]["title"])
        all_headlines.append(headlines)
        all_dates.append(date)
        date = date - timedelta(days=1)
        all_count = articles['totalResults']

    art_sentiments = []

    for article in articles["articles"]:
        try:
            text = article["content"]
            date = article["publishedAt"][:10]
            sentiment = analyzer.polarity_scores(text)
            compound = sentiment["compound"]
            pos = sentiment["pos"]
            neu = sentiment["neu"]
            neg = sentiment["neg"]
        
            art_sentiments.append({
                "text": text,
                "date": date,
                "compound": compound,
                "positive": pos,
                "negative": neg,
                "neutral": neu
            
            })
        
        except AttributeError:
            pass
    art_df= pd.DataFrame(art_sentiments)
    # Reorder DataFrame columns
    cols = ["date", "text", "compound", "positive", "negative", "neutral"]
    art_df = art_df[cols]
    return all_headlines, all_dates, all_count, art_df, articles

In [17]:
# Fetch the Bitcoin news articles
btc_headlines, btc_dates, count, btc_df = get_headlines("bitcoin")
print(count)

Fetching news about 'bitcoin'
******************************
retrieving news from: 2022-02-10 00:00:00
retrieving news from: 2022-02-09 00:00:00
retrieving news from: 2022-02-08 00:00:00
retrieving news from: 2022-02-07 00:00:00
retrieving news from: 2022-02-06 00:00:00
retrieving news from: 2022-02-05 00:00:00
retrieving news from: 2022-02-04 00:00:00
retrieving news from: 2022-02-03 00:00:00
retrieving news from: 2022-02-02 00:00:00
retrieving news from: 2022-02-01 00:00:00
retrieving news from: 2022-01-31 00:00:00
retrieving news from: 2022-01-30 00:00:00
retrieving news from: 2022-01-29 00:00:00
retrieving news from: 2022-01-28 00:00:00
retrieving news from: 2022-01-27 00:00:00
retrieving news from: 2022-01-26 00:00:00
retrieving news from: 2022-01-25 00:00:00
retrieving news from: 2022-01-24 00:00:00
retrieving news from: 2022-01-23 00:00:00
retrieving news from: 2022-01-22 00:00:00
retrieving news from: 2022-01-21 00:00:00
retrieving news from: 2022-01-20 00:00:00
retrieving news

In [76]:
# Fetch the Ethereum news articles
eth_headlines, eth_dates = get_headlines("ethereum")

Fetching news about 'ethereum'
******************************
retrieving news from: 2022-02-09 00:00:00


NewsAPIException: {'status': 'error', 'code': 'rateLimited', 'message': 'You have made too many requests recently. Developer accounts are limited to 100 requests over a 24 hour period (50 requests available every 12 hours). Please upgrade to a paid plan if you need more requests.'}

In [20]:
# Instantiate SentimentIntensityAnalyzer
sid = SentimentIntensityAnalyzer()

In [21]:
len(btc_headlines[0])

20

In [22]:
btc_df.head()

,date,text,compound,positive,negative,neutral
0,2022-01-12,When Denis Rusinovich set up cryptocurrency mi...,0.0000,0.000,0.00,1.000
1,2022-01-12,So a single person wrote a proposal and this i...,0.0000,0.000,0.00,1.000
2,2022-01-12,"NEW YORK, Jan 12 (Reuters) - Softbank-backed B...",0.0000,0.000,0.00,1.000
3,2022-01-12,Block CEO Jack Dorsey has revealed plans to cr...,0.5267,0.226,0.07,0.704
4,2022-01-12,"They say time is money, but if that’s true, mo...",0.5719,0.089,0.00,0.911


In [ ]:
eth_df.head()

In [23]:
# Describe the Bitcoin Sentiment
btc_df.describe()

,compound,positive,negative,neutral
count,20.000000,20.000000,20.000000,20.000000
mean,0.190435,0.066700,0.025250,0.908050
std,0.301348,0.069007,0.037179,0.081381
min,-0.401900,0.000000,0.000000,0.704000
25%,0.000000,0.000000,0.000000,0.860500
50%,0.152950,0.061500,0.000000,0.908500
75%,0.405525,0.086750,0.058000,1.000000
max,0.735100,0.226000,0.113000,1.000000


In [ ]:
# Describe the Ethereum Sentiment
eth_df.describe()

### Questions:

Q: Which coin had the highest mean positive score?

A: 

Q: Which coin had the highest compound score?

A: 

Q. Which coin had the highest positive score?

A: 

---

## 2. Natural Language Processing
---
###   Tokenizer

In this section, you will use NLTK and Python to tokenize the text for each coin. Be sure to:
1. Lowercase each word.
2. Remove Punctuation.
3. Remove Stopwords.

In [24]:
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
from string import punctuation
import re

In [25]:
# Instantiate the lemmatizer
lemmatizer = WordNetLemmatizer()

# Create a list of stopwords
sw = set(stopwords.words('english'))

# Expand the default stopwords list if necessary
# YOUR CODE HERE!

In [28]:
# Complete the tokenizer function
def tokenizer(text):
    """Tokenizes text."""
    # Remove the punctuation from text
    regex = re.compile("[^a-zA-Z ]")
    # Create a tokenized list of the words
    re_clean = regex.sub('', articles)
    words = word_tokenize(re_clean)
    # Lemmatize words into root words
    lem = [lemmatizer.lemmatize(word) for word in words]
    # Convert the words to lowercase & Remove the stop words
    output = [word.lower() for word in lem if word.lower() not in sw]
    return tokens



In [29]:
# Create a new tokens column for Bitcoin
print(set(tokenizer(btc_headlines)))


NameError: name 'articles' is not defined

In [81]:
# Create a new tokens column for Ethereum
print(set(tokenizer(etc_headlines)))

SyntaxError: unexpected EOF while parsing (Temp/ipykernel_28000/39349498.py, line 2)

---

### NGrams and Frequency Analysis

In this section you will look at the ngrams and word frequency for each coin. 

1. Use NLTK to produce the n-grams for N = 2. 
2. List the top 10 words for each coin. 

In [ ]:
from collections import Counter
from nltk import ngrams

In [ ]:
# Generate the Bitcoin N-grams where N=2
# YOUR CODE HERE!

In [ ]:
# Generate the Ethereum N-grams where N=2
# YOUR CODE HERE!

In [ ]:
# Function token_count generates the top 10 words for a given coin
def token_count(tokens, N=3):
    """Returns the top N tokens from the frequency count"""
    return Counter(tokens).most_common(N)

In [ ]:
# Use token_count to get the top 10 words for Bitcoin
# YOUR CODE HERE!

In [ ]:
# Use token_count to get the top 10 words for Ethereum
# YOUR CODE HERE!

---

### Word Clouds

In this section, you will generate word clouds for each coin to summarize the news for each coin

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = [20.0, 10.0]

In [ ]:
# Generate the Bitcoin word cloud
# YOUR CODE HERE!

In [ ]:
# Generate the Ethereum word cloud
# YOUR CODE HERE!

---
## 3. Named Entity Recognition

In this section, you will build a named entity recognition model for both Bitcoin and Ethereum, then visualize the tags using SpaCy.

In [ ]:
import spacy
from spacy import displacy

In [ ]:
# Download the language model for SpaCy
# !python -m spacy download en_core_web_sm

In [ ]:
# Load the spaCy model
nlp = spacy.load('en_core_web_sm')

---
### Bitcoin NER

In [ ]:
# Concatenate all of the Bitcoin text together
# YOUR CODE HERE!

In [ ]:
# Run the NER processor on all of the text
# YOUR CODE HERE!

# Add a title to the document
# YOUR CODE HERE!

In [ ]:
# Render the visualization
# YOUR CODE HERE!

In [ ]:
# List all Entities
# YOUR CODE HERE!

---

### Ethereum NER

In [ ]:
# Concatenate all of the Ethereum text together
# YOUR CODE HERE!

In [ ]:
# Run the NER processor on all of the text
# YOUR CODE HERE!

# Add a title to the document
# YOUR CODE HERE!

In [ ]:
# Render the visualization
# YOUR CODE HERE!

In [ ]:
# List all Entities
# YOUR CODE HERE!

---